## TRADE DATA

In [276]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_sunflower_oil_0 = process_csv("trade_data/sunflower_oil.csv", 0)
df_sunflower_oil_0['price'] = df_sunflower_oil_0['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_0['OBS_VALUE_QUANTITY_TON']

df_sunflower_oil_1 = process_csv("trade_data/sunflower_oil.csv", 1)
df_sunflower_oil_1['price'] = df_sunflower_oil_1['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_1['OBS_VALUE_QUANTITY_TON']

df_sunflower = df_sunflower_oil_0.merge(df_sunflower_oil_1, on=["PARTNER_Labels", "TIME_PERIOD"], how='outer')
df_sunflower = df_sunflower.fillna(0)

df_sunflower['OBS_VALUE_QUANTITY_TON'] = df_sunflower['OBS_VALUE_QUANTITY_TON_x'] + df_sunflower['OBS_VALUE_QUANTITY_TON_y']
df_sunflower['OBS_VALUE_VALUE_1000EURO'] = df_sunflower['OBS_VALUE_VALUE_1000EURO_x'] + df_sunflower['OBS_VALUE_VALUE_1000EURO_y']
df_sunflower['price'] = df_sunflower['OBS_VALUE_VALUE_1000EURO'] / df_sunflower['OBS_VALUE_QUANTITY_TON']

df_global_temperature['TIME_PERIOD']

0       2005-01-01
1       2005-02-01
2       2005-03-01
3       2005-04-01
4       2005-05-01
           ...    
21085   2022-08-01
21086   2022-09-01
21087   2022-10-01
21088   2022-11-01
21089   2022-12-01
Name: TIME_PERIOD, Length: 21090, dtype: datetime64[ns]

In [277]:
# group by TIME_PERIOD and aggregate the OBS_VALUE_QUANTITY_TON and OBS_VALUE_VALUE_1000EURO columns by summing
grouped_df = df_sunflower.groupby('TIME_PERIOD').agg({'OBS_VALUE_QUANTITY_TON': 'sum', 'OBS_VALUE_VALUE_1000EURO': 'sum'})

# calculate globalprice
globalprice = pd.DataFrame(grouped_df['OBS_VALUE_VALUE_1000EURO'] / grouped_df['OBS_VALUE_QUANTITY_TON'])
# rename the column to 'globalprice'
globalprice = globalprice.rename(columns={0: 'globalprice'})


globalprice = globalprice.reset_index()
globalprice = globalprice.rename(columns={'index': 'TIME_PERIOD'})

globalprice

,TIME_PERIOD,globalprice
0,2005-01-01,0.547673
1,2005-02-01,0.544328
2,2005-03-01,0.561141
3,2005-04-01,0.530829
4,2005-05-01,0.511798
...,...,...
209,2022-06-01,1.535339
210,2022-07-01,1.841942
211,2022-08-01,1.759469
212,2022-09-01,1.613330


## WEATHER DATA

In [278]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature['TIME_PERIOD']

0       2005-01-01
1       2005-02-01
2       2005-03-01
3       2005-04-01
4       2005-05-01
           ...    
21085   2022-08-01
21086   2022-09-01
21087   2022-10-01
21088   2022-11-01
21089   2022-12-01
Name: TIME_PERIOD, Length: 21090, dtype: datetime64[ns]

## OIL DATA

In [279]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price['TIME_PERIOD']

540   2005-01-01
541   2005-02-01
542   2005-03-01
543   2005-04-01
544   2005-05-01
         ...    
749   2022-06-01
750   2022-07-01
751   2022-08-01
752   2022-09-01
753   2022-10-01
Name: TIME_PERIOD, Length: 214, dtype: datetime64[ns]

## FUTURES PRICES (SUNFLOWER OIL)

In [280]:
# Extracted from https://www.investing.com/commodities/high-oil-content-sunflower-seed-historical-data
import pandas as pd
import datetime

#Clean import High Oil Content Sunflower Seed Futures Historical Data.csv

# Read the data from the file
df_futures_sunflower_seed = pd.read_csv('futures_data/High Oil Content Sunflower Seed Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_sunflower_seed.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Sunflower Seed Price Futures'}, inplace=True)

df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_sunflower_seed['TIME_PERIOD'] = df_futures_sunflower_seed['TIME_PERIOD'].dt.strftime('%Y/%d/%m')

df_futures_sunflower_seed.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_sunflower_seed.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)

# Replace commas with dots in 'Sunflower Seed Price Futures' column
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].str.replace(',', '.')

# Convert 'Sunflower Seed Price Futures' column to float datatype
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].astype(float)

# convert TIME_PERIOD column to datetime format year-month-day
df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'], format='%Y/%m/%d')

df_futures_sunflower_seed

,TIME_PERIOD,Sunflower Seed Price Futures
138,2006-10-01,49.2
137,2006-11-01,52.0
136,2006-12-01,54.5
135,2007-01-01,54.5
134,2007-02-01,54.5
...,...,...
4,2022-06-01,267.0
3,2022-07-01,262.0
2,2022-08-01,265.0
1,2022-09-01,267.0


## MACROECONOMIC DATA

In [7]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


In [8]:
import pandas as pd
import datetime
import numpy as np
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

#delete the last 5 empty rows 
num_rows = len(population_data)
population_data = population_data.drop(population_data.index[num_rows-5:num_rows])
#rename a column 
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels','Time':'TIME_PERIOD'})
population_data = population_data.rename(columns = {'Population, total [SP.POP.TOTL]': 'A'})

#get's the index s of the columns with string
df = population_data
string_indices = df['A'][df['A'].apply(lambda x: isinstance(x, str))].index

#gets th countries corresponging to the index s 
lll = len(string_indices)
LIST_of_del_countries = [None] * lll

counter = 0 
for i in range(lll):
    LIST_of_del_countries[counter] = population_data.iloc[(string_indices[i]),0]
    counter = counter +1

list1 = list(set(LIST_of_del_countries))

print(df.shape)
print(list1)
for j in range(4000):
  
    if df.iloc[j,0] in list1:
        df = df.drop(index=[j])
        print(df.shape)

df = df.replace('..', 0)
print(df)
# boolean indexing to select rows to delete
idx = df['PARTNER_Labels'].str.contains('IDA blend')

# drop rows using boolean indexing
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA only')
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA total')
df = df.drop(df[idx].index)

for name in list1:
    idx = df['PARTNER_Labels'].str.contains(name)
    df = df.drop(df[idx].index)


testing= df.copy()
excel_test = pd.ExcelWriter("population_data.xlsx")
testing.to_excel(excel_test)
excel_test.save()
#make the 
monthly_data_p = population_data.iloc[0:0].copy()

number_of_col = df.shape[1]
nan_row = np.empty((number_of_col,))
nan_row[:] = np.nan

n_row = df.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = df.iloc[i,:]
    current_country = df.iloc[i,0]
    current_country_code = df.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data_p.loc[index_3] = holder_for_rows   
        else:
            monthly_data_p.loc[index_3] = nan_row
            monthly_data_p.iloc[index_3,0] = current_country
            monthly_data_p.iloc[index_3,1] = current_country_code 


# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper2.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()

c_p = population_data.columns[4]
monthly_data_p[c_p] = monthly_data_p[c_p].apply(pd.to_numeric)
n_countries = len(df['Country Code'].unique())
#fills in the date column
for i in range(n_countries-10):
    index_1 = 0   + 228*i
    index_2 = 228 + 228*i
    monthly_data_p.iloc[index_1:index_2,4] = monthly_data_p.iloc[index_1:index_2,4].interpolate().copy()
    #adding dates 
    monthly_data_p.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2023-12", freq='M')

# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper3.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()


(5054, 5)
['IDA blend', 'St. Martin (French part)', 'Not classified', 'Greenland', 'Bermuda', 'Sint Maarten (Dutch part)', 'IBRD only', 'IDA total', 'IDA only']
(5053, 5)
(5052, 5)
(5051, 5)
(5050, 5)
(5049, 5)
(5048, 5)
(5047, 5)
(5046, 5)
(5045, 5)
(5044, 5)
(5043, 5)
(5042, 5)
(5041, 5)
(5040, 5)
(5039, 5)
(5038, 5)
(5037, 5)
(5036, 5)
(5035, 5)
(5034, 5)
(5033, 5)
(5032, 5)
(5031, 5)
(5030, 5)
(5029, 5)
(5028, 5)
(5027, 5)
(5026, 5)
(5025, 5)
(5024, 5)
(5023, 5)
(5022, 5)
(5021, 5)
(5020, 5)
(5019, 5)
(5018, 5)
(5017, 5)
(5016, 5)
(5015, 5)
(5014, 5)
(5013, 5)
(5012, 5)
(5011, 5)
(5010, 5)
(5009, 5)
(5008, 5)
(5007, 5)
(5006, 5)
(5005, 5)
(5004, 5)
(5003, 5)
(5002, 5)
(5001, 5)
(5000, 5)
(4999, 5)
(4998, 5)
(4997, 5)
(4996, 5)
(4995, 5)
(4994, 5)
(4993, 5)
(4992, 5)
(4991, 5)
(4990, 5)
(4989, 5)
(4988, 5)
(4987, 5)
(4986, 5)
(4985, 5)
(4984, 5)
(4983, 5)
(4982, 5)
(4981, 5)
(4980, 5)
(4979, 5)
(4978, 5)
(4977, 5)
(4976, 5)
(4975, 5)
     PARTNER_Labels Country Code  TIME_PERIOD Tim

C:\Users\basbe\AppData\Local\Temp\ipykernel_13872\3099761979.py:54: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df['PARTNER_Labels'].str.contains(name)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066


3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707


In [9]:
monthly_data_p  = monthly_data_p.rename(columns = {'Country Code': 'country_code'})
# convert Period and float values to string format, and then to datetime
monthly_data_p ['TIME_PERIOD'] = monthly_data_p ['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data_p ['TIME_PERIOD'] = pd.to_datetime(monthly_data_p ['TIME_PERIOD'], errors='coerce')
monthly_data_p.head(214)

,PARTNER_Labels,country_code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06-01,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07-01,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08-01,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09-01,NaN,4.186949e+07


In [10]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

In [11]:
merged_monthly_data_df = monthly_data.merge(monthly_data_p, on=['country_code', 'TIME_PERIOD'], how='outer')
merged_monthly_data_df = merged_monthly_data_df.drop('PARTNER_Labels_y', axis=1)
merged_monthly_data_df = merged_monthly_data_df.rename(columns={'PARTNER_Labels_x': 'PARTNER_Labels', 'A': 'population_projections'})


In [13]:
df_macroeconomic = merged_monthly_data_df
df_macroeconomic

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,Argentina,ARG,2005-01-01 00:00:00.000000000,1.987371e+11,4.352966e+11,39070501.0,11.510000,1.299209e+06,9.640000,1712.850000,89.050000,YR2005,39070501.0
1,Argentina,ARG,2005-02-01 00:00:00.000000000,2.015554e+11,4.382157e+11,39104363.5,11.390833,1.299391e+06,9.601667,1723.801667,89.016667,NaN,39104363.5
2,Argentina,ARG,2005-03-01 00:00:00.000000000,2.043738e+11,4.411348e+11,39138226.0,11.271667,1.299572e+06,9.563333,1734.753333,88.983333,NaN,39138226.0
3,Argentina,ARG,2005-04-01 00:00:00.000000000,2.071921e+11,4.440538e+11,39172088.5,11.152500,1.299754e+06,9.525000,1745.705000,88.950000,NaN,39172088.5
4,Argentina,ARG,2005-05-01 00:00:00.000000000,2.100105e+11,4.469729e+11,39205951.0,11.033333,1.299936e+06,9.486667,1756.656667,88.916667,NaN,39205951.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61747,NaN,ZWE,1970-01-01 00:00:00.000002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2019,15354608.0
61748,NaN,ZWE,1970-01-01 00:00:00.000002020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2020,15669666.0
61749,NaN,ZWE,1970-01-01 00:00:00.000002021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2021,15993524.0
61750,NaN,ZWE,1970-01-01 00:00:00.000002022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2022,16320537.0


## MERGING DATA

### Start with empty dataframe with all possible combinations

In [281]:
import itertools
import pandas as pd

df_sunflower['TIME_PERIOD'] = pd.to_datetime(df_sunflower['TIME_PERIOD'])

# Get all unique values for TIME_PERIOD and PARTNER_Labels
time_periods = df_sunflower['TIME_PERIOD'].unique()
partner_labels = df_sunflower['PARTNER_Labels'].unique()

# Generate all possible combinations
combinations = list(itertools.product(time_periods, partner_labels))

# Create a new dataframe with the combinations
df_combinations = pd.DataFrame(combinations, columns=['TIME_PERIOD', 'PARTNER_Labels'])

# Merge with the original dataframe to get the corresponding values
df_result = pd.merge(df_combinations, df_sunflower, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')

# Fill any missing values with 0
df_result = df_result.fillna(0)
df_result

,TIME_PERIOD,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,2005-01-01,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478
1,2005-01-01,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
2,2005-01-01,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
3,2005-01-01,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877
4,2005-01-01,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18613,2021-04-01,Taiwan,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
18614,2021-04-01,Tanzania,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
18615,2021-04-01,Tunisia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
18616,2021-04-01,Uganda,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000


## Merge with global price data

In [282]:
df_result['TIME_PERIOD'] = pd.to_datetime(df_result['TIME_PERIOD'])
globalprice['TIME_PERIOD'] = pd.to_datetime(globalprice['TIME_PERIOD'])

# add globalprice as a new column to the dataframe
df_merged_result_globalprice = df_result.merge(globalprice, on=['TIME_PERIOD'], how='right')
df_merged_result_globalprice = df_merged_result_globalprice.rename(columns={"globalprice_y": "globalprice"})

df_merged_result_globalprice

,TIME_PERIOD,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,0.547673
1,2005-01-01,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
2,2005-01-01,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
3,2005-01-01,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673
4,2005-01-01,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18613,2022-10-01,Taiwan,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,1.477840
18614,2022-10-01,Tanzania,0.0,0.0,0.00,0.000000,15121990.0,0.3,0.32,1.066667,0.3,0.32,1.066667,1.477840
18615,2022-10-01,Tunisia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,1.477840
18616,2022-10-01,Uganda,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.10,1.000000,0.1,0.10,1.000000,1.477840


### Merged trade data, oil price

In [283]:
df_merged_trade_oil = df_oil_price.merge(df_merged_result_globalprice, on='TIME_PERIOD', how='outer')
df_merged_trade_oil

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,0.547673
1,2005-01-01,42.972278,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
2,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673
4,2005-01-01,42.972278,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18613,2022-10-01,90.326667,Taiwan,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,1.477840
18614,2022-10-01,90.326667,Tanzania,0.0,0.0,0.00,0.000000,15121990.0,0.3,0.32,1.066667,0.3,0.32,1.066667,1.477840
18615,2022-10-01,90.326667,Tunisia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,1.477840
18616,2022-10-01,90.326667,Uganda,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.10,1.000000,0.1,0.10,1.000000,1.477840


### Merged trade data, oil price, weather data

In [284]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,0.547673,NaN,NaN,NaN
1,2005-01-01,42.972278,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,242.290980,162.670330,324.125000
2,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,-1.166667,-172.500000,125.000000
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673,30.700000,-31.000000,79.000000
4,2005-01-01,42.972278,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2022-08-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.060676,244.363636,348.666667
24651,2022-09-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279.751352,233.600000,350.000000
24652,2022-10-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.000000,238.000000,260.000000
24653,2022-11-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.000000,252.000000,304.000000


### Merged trade data, oil price, weather, future prices data

In [285]:
df_merged_trade_oil_weather['TIME_PERIOD'] = pd.to_datetime(df_merged_trade_oil_weather['TIME_PERIOD'] , format='%Y/%m/%d')

df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_sunflower_seed, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures.head(4000)

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Sunflower Seed Price Futures
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,0.547673,NaN,NaN,NaN,NaN
1,2005-01-01,42.972278,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,242.290980,162.670330,324.125000,NaN
2,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,-1.166667,-172.500000,125.000000,NaN
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673,30.700000,-31.000000,79.000000,NaN
4,2005-01-01,42.972278,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2007-12-01,89.519942,Denmark,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.655821,46.599462,1.500000,145.000000,85.9
3996,2007-12-01,89.519942,Egypt,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.655821,153.903998,54.222222,253.000000,85.9
3997,2007-12-01,89.519942,Estonia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.655821,20.961716,-64.769231,69.000000,85.9
3998,2007-12-01,89.519942,Finland,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.655821,-7.834590,-100.529412,45.466667,85.9


### Merged trade, oil price, weather, future prices, macroeconomic 

In [290]:
df_macroeconomic['TIME_PERIOD'] = pd.to_datetime(df_macroeconomic['TIME_PERIOD'] , format='%Y/%m/%d')

df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged



,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Sunflower Seed Price Futures,country_code,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,0.547673,NaN,NaN,NaN,NaN,ARG,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
1,2005-01-01,42.972278,Australia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,242.290980,162.670330,324.125000,NaN,AUS,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,20176844.0
2,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,-1.166667,-172.500000,125.000000,NaN,AUT,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673,30.700000,-31.000000,79.000000,NaN,BEL,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
4,2005-01-01,42.972278,Bolivia,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.547673,NaN,NaN,NaN,NaN,BOL,9.549078e+09,2.018353e+10,9377388.0,5.45,369620.0,23.13,553.92,83.27,YR2005,9377388.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24651,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24652,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24653,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CLEANING EXTREME VALUES

In [291]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)
df_merged = df_merged.replace(0, np.nan)
df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Sunflower Seed Price Futures,country_code,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,NaN,NaN,NaN,NaN,21086.3,10489.97,0.497478,0.547673,NaN,NaN,NaN,NaN,ARG,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
1,2005-01-01,42.972278,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,242.290980,162.670330,324.125000,NaN,AUS,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,20176844.0
2,2005-01-01,42.972278,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,-1.166667,-172.500000,125.000000,NaN,AUT,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673,30.700000,-31.000000,79.000000,NaN,BEL,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
4,2005-01-01,42.972278,Bolivia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,NaN,NaN,NaN,NaN,BOL,9.549078e+09,2.018353e+10,9377388.0,5.45,369620.0,23.13,553.92,83.27,YR2005,9377388.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24651,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24652,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24653,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total', 'unemployment_total', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)

df_merged


,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Sunflower Seed Price Futures,country_code,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,NaN,NaN,NaN,NaN,21086.3,10489.97,0.497478,0.547673,NaN,NaN,NaN,NaN,ARG,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
1,2005-01-01,42.972278,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,242.290980,162.670330,324.125000,NaN,AUS,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,20176844.0
2,2005-01-01,42.972278,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,-1.166667,-172.500000,125.000000,NaN,AUT,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
3,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,0.547673,30.700000,-31.000000,79.000000,NaN,BEL,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
4,2005-01-01,42.972278,Bolivia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547673,NaN,NaN,NaN,NaN,BOL,9.549078e+09,2.018353e+10,9377388.0,5.45,369620.0,23.13,553.92,83.27,YR2005,9377388.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24651,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24652,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24653,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## FULL DATAFRAME (without filtering top countries)

In [293]:
pd.set_option('display.max_columns', 2000) #replace n with the number of columns you want to see completely
###FOR NOW
df_sunflower_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['globalprice', 'OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Sunflower Seed Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=False)
df_sunflower_total['price']


PARTNER_Labels,Argentina,Armenia,Australia,Austria,Belgium,Bolivia,Bosnia and Herzegovina,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,Cyprus,Czechia,Côte d'Ivoire,Denmark,Egypt,Estonia,Ethiopia,Finland,France,Germany,Ghana,Greece,Hungary,India,Indonesia,Ireland,Israel,Italy,Japan,Kazakhstan,Kenya,Kosovo,Latvia,Lebanon,Lithuania,Luxembourg,Malaysia,Malta,Mexico,Moldova,Morocco,Mozambique,Namibia,Nigeria,North Macedonia,Norway,Oman,Pakistan,Paraguay,Peru,Philippines,Poland,Portugal,Romania,Russia,Saudi Arabia,Senegal,Serbia,Singapore,Slovakia,Slovenia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Tanzania,Thailand,Tunisia,Türkiye,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.497478,NaN,NaN,NaN,0.754877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.658400,NaN,NaN,NaN,NaN,0.652793,0.777966,NaN,NaN,NaN,NaN,NaN,0.780650,NaN,0.935455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.4,0.580502,NaN,NaN,2.560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-01,0.488180,NaN,NaN,NaN,0.741206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.733961,0.768516,NaN,NaN,0.998305,NaN,NaN,0.788262,1.0,0.862326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.579589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.602395,NaN,NaN
2005-03-01,0.496523,NaN,NaN,9.900000,0.746156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.681311,0.779057,NaN,NaN,NaN,NaN,NaN,0.828878,NaN,0.961020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.771633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.541757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.887970,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-01,0.492487,NaN,NaN,NaN,0.707116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.646057,0.795872,NaN,NaN,NaN,NaN,NaN,0.847345,NaN,0.821429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.725490,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-01,0.485282,NaN,NaN,NaN,0.724853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.658766,0.712101,NaN,NaN,NaN,NaN,NaN,0.816734,NaN,1.006273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.778431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,1.581752,NaN,NaN,1.267025,2.131095,NaN,NaN,NaN,2.055386,NaN,1.217647,1.6,1.2175,1.550292,1.3,1.376659,NaN,1.326582,NaN,1.341667,NaN,1.291667,1.920146,1.680164,NaN,1.422148,1.665363,NaN,NaN,1.318333,NaN,1.467910,1.325,NaN,NaN,NaN,1.261017,NaN,1.374468,1.319512,0.7,NaN,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679370,1.341129,1.365285,1.30,NaN,NaN,1.750277,NaN,1.344828,1.2875,NaN,2.013309,NaN,1.600941,1.212414,NaN,NaN,1.300000,NaN,NaN,2.368726,NaN,1.616470,NaN,2.601862,1.251235,NaN
2022-09-01,1.157143,NaN,NaN,1.299674,2.153551,NaN,NaN,NaN,2.362470,NaN,NaN,NaN,NaN,1.902422,1.3,1.287361,NaN,1.318065,NaN,1.262069,NaN,1.324194,1.651856,1.677523,NaN,1.450692,1.146126,NaN,NaN,1.351613,NaN,1.452788,NaN,NaN,NaN,NaN,1.300000,NaN,1.346154,1.276000,NaN,1.3,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,1.300000,NaN,NaN,1.460022,1.372593,1.353968,1.40,NaN,NaN,1.516750,NaN,1.302273,1.3200,NaN,1.281950,N

## HANDLE MISSING DATA

### Fill in missing values with global price

In [294]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_sunflower_total[['price']].loc[:, ('price', df_sunflower_total['price'].isna().any())]
df_missing_price = df_missing_price.iloc[:-2]
# Forward fill any remaining missing values
df_missing_price['price'] = df_missing_price['price'].fillna(method='ffill')
# Back fill any missing values at the beginning of the series
df_missing_price['price'] = df_missing_price['price'].fillna(method='bfill')


In [295]:
###FOR NOW
#df_sunflower_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=True)

# Create a new DataFrame that only contains the globalprice
df_price = globalprice['globalprice']
df_price = pd.DataFrame(df_price)
df_price.set_index(globalprice['TIME_PERIOD'], inplace=True)

# Calculate the year-over-year percent change in 'price' for France and Germany:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)

df_price_pct_change = pd.DataFrame(df_price_pct_change)
df_price_pct_change

,globalprice
TIME_PERIOD,
2005-01-01,0.000000
2005-02-01,-0.006109
2005-03-01,0.030889
2005-04-01,-0.054019
2005-05-01,-0.035852
...,...
2022-06-01,0.147555
2022-07-01,0.199697
2022-08-01,-0.044775


In [296]:
multiplyfactor = 1 + df_price_pct_change['globalprice']
multiplyfactor = pd.DataFrame(multiplyfactor)

df_result1 = df_missing_price.multiply(multiplyfactor['globalprice'], axis='index')

mask = df_sunflower_total['price'].isna()

df_sunflower_total_updated = df_sunflower_total.combine_first(df_result1[mask])
df_sunflower_total_updated['price']

PARTNER_Labels,Argentina,Armenia,Australia,Austria,Belgium,Bolivia,Bosnia and Herzegovina,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,Cyprus,Czechia,Côte d'Ivoire,Denmark,Egypt,Estonia,Ethiopia,Finland,France,Germany,Ghana,Greece,Hungary,India,Indonesia,Ireland,Israel,Italy,Japan,Kazakhstan,Kenya,Kosovo,Latvia,Lebanon,Lithuania,Luxembourg,Malaysia,Malta,Mexico,Moldova,Morocco,Mozambique,Namibia,Nigeria,North Macedonia,Norway,Oman,Pakistan,Paraguay,Peru,Philippines,Poland,Portugal,Romania,Russia,Saudi Arabia,Senegal,Serbia,Singapore,Slovakia,Slovenia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Tanzania,Thailand,Tunisia,Türkiye,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.497478,2.500000,1.979911,9.900000,0.754877,1.008690,0.700000,0.700000,0.900000,4.566667,0.715860,12.386290,1.008333,1.100000,1.694681,1.224442,0.700000,0.658400,1.094444,1.366667,1.478049,3.416667,0.652793,0.777966,1.212195,1.400000,0.998305,1.188889,2.100000,0.780650,1.000000,0.935455,0.477941,0.800000,0.700000,1.500000,0.912683,3.200000,0.834842,1.559091,0.700000,1.150000,1.929148,0.485984,1.128571,1.300000,1.447368,0.600000,0.381765,0.577173,1.000000,0.500000,0.771633,0.708716,0.886720,0.204000,1.220000,0.778862,0.519589,0.885602,0.700000,0.681660,0.800000,0.606224,0.929713,3.400000,0.580502,0.925000,0.705556,2.560000,1.428571,1.300000,1.400000,0.700000,0.700000,0.887970,1.100000,0.457111,1.250000,0.602395,0.482400,0.779487
2005-02-01,0.488180,2.484728,1.967816,9.839523,0.741206,1.002528,0.695724,0.695724,0.894502,4.538770,0.711487,12.310625,1.002174,1.093280,1.684328,1.216962,0.695724,0.654378,1.087759,1.358318,1.469020,3.395795,0.733961,0.768516,1.204790,1.391448,0.998305,1.181626,2.087171,0.788262,1.000000,0.862326,0.475022,0.795113,0.695724,1.490837,0.907108,3.180452,0.829742,1.549567,0.695724,1.142975,1.917363,0.483016,1.121677,1.292059,1.438527,0.596335,0.379433,0.573647,0.993891,0.496946,0.766919,0.704386,0.881303,0.202754,1.212547,0.774104,0.516415,0.880192,0.695724,0.677495,0.795113,0.602521,0.924034,3.379230,0.579589,0.919349,0.701245,2.544361,1.419845,1.292059,1.391448,0.695724,0.695724,0.882545,1.093280,0.454319,1.242364,0.602395,0.479453,0.774725
2005-03-01,0.496523,2.577221,2.041067,9.900000,0.746156,1.039847,0.721622,0.721622,0.927800,4.707725,0.737972,12.768885,1.039479,1.133977,1.747027,1.262263,0.721622,0.678737,1.128250,1.408881,1.523704,3.522203,0.681311,0.779057,1.249638,1.443244,1.029141,1.225612,2.164866,0.828878,1.030889,0.961020,0.492704,0.824711,0.721622,1.546333,0.940874,3.298843,0.860629,1.607249,0.721622,1.185522,1.988737,0.500996,1.163431,1.340155,1.492076,0.618533,0.393558,0.595001,1.030889,0.515444,0.771633,0.730607,0.914110,0.210301,1.257684,0.802920,0.535639,0.912957,0.721622,0.702715,0.824711,0.624949,0.958431,3.505021,0.541757,0.953572,0.727349,2.639075,1.472698,1.340155,1.443244,0.721622,0.721622,0.887970,1.133977,0.471231,1.288611,0.621002,0.497301,0.803564
2005-04-01,0.492487,2.364953,1.872959,9.365215,0.707116,0.954202,0.662187,0.662187,0.851383,4.319981,0.677190,11.717199,0.953864,1.040579,1.603136,1.158299,0.662187,0.622834,1.035324,1.292841,1.398207,3.232103,0.646057,0.795872,1.146714,1.324374,0.944378,1.124667,1.986561,0.847345,0.945981,0.821429,0.452123,0.756785,0.662187,1.418972,0.863381,3.027140,0.789745,1.474871,0.662187,1.087878,1.824938,0.459732,1.067607,1.229776,1.369183,0.567589,0.361143,0.545995,0.945981,0.472991,0.729950,0.670432,0.838821,0.192980,1.154097,0.736789,0.491522,0.837763,0.662187,0.644837,0.756785,0.573477,0.879491,3.216336,0.549371,0.875033,0.667442,2.421712,1.351402,1.229776,1.324374,0.662187,0.662187,0.725490,1.040579,0.432419,1.182477,0.569854,0.456341,0.737380
2005-05-01,0.485282,2.410369,1.908926,9.545061,0.724853,0.972526,0.674903,0.674903,0.867733,4.402940,0.690194,11.942211,0.972182,1.060562,1.633922,1.180543,0.6

### Interpolate macroeconomic

In [297]:
# fill missing values at the beginning of each column with the first known value
df_sunflower_total_updated.fillna(method='ffill')

# fill missing values at the end of each column with the last known value
df_sunflower_total_updated.fillna(method='bfill')

# interpolate missing values for selected columns
cols_to_interpolate = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

for col in cols_to_interpolate:
    df_sunflower_total_updated.loc[:, (col, slice(None))] = df_sunflower_total.loc[:, (col, slice(None))].interpolate()
    df_sunflower_total_updated.fillna(method='bfill')
    df_sunflower_total_updated.fillna(method='ffill')

# print the updated pivot table dataframe
df_sunflower_total_updated


AVG_TAVG                                             \
PARTNER_Labels     Albania     Algeria Angola     Armenia   Australia   
TIME_PERIOD                                                             
2005-01-01             NaN   76.228202  274.0  -87.478261  242.290980   
2005-02-01             NaN   84.478184  270.5  -87.000000  237.755531   
2005-03-01             NaN  147.382230  156.0  -46.440000  223.480383   
2005-04-01             NaN  181.541272  242.5   39.714286  208.498275   
2005-05-01             NaN  240.048870  211.5   96.043478  167.861508   
...                    ...         ...    ...         ...         ...   
2022-08-01      144.666667  312.218029  213.0  209.490002  146.200527   
2022-09-01      144.666667  286.008954  213.0  173.108906  165.721311   
2022-10-01      144.666667  231.950455  213.0  111.054056  189.637400   
2022-11-01      144.666667  168.537960  213.0   50.784508  201.174639   
2022-12-01      144.666667  150.914659  213.0   -1.926037  224.700188   

                                                                            \
PARTNER_Labels     Austria  Azerbaijan     Bahrain     Belarus     Belgium   
TIME_PERIOD                                                                  
2005-01-01       -1.166667         NaN  160.222222   -7.788025   30.700000   
2005-02-01      -42.166667   72.000000  176.000000  -59.820327   20.000000   
2005-03-01       -1.994624   68.000000  211.095238  -32.497550   55.800000   
2005-04-01       60.233333  191.000000  257.315789   80.398109  111.000000   
2005-05-01      108.758065  200.000000  303.117647  130.261818  129.250000   
...                    ...         ...         ...         ...         ...   
2022-08-01      175.258849  260.574138  360.153846  207.011045  213.200000   
2022-09-01      103.779082  218.628333  346.350000   99.815254  166.000000   
2022-10-01       98.433871  160.495556  307.000000   82.525416  147.333333   
2022-11-01       36.939583   96.659524  274.357143   17.335752   88.454545   
2022-12-01       -9.296552   37.345238  223.176471  -29.105331   36.900000   

                                                                            \
PARTNER_Labels Bosnia and Herzegovina      Brazil    Bulgaria Burkina Faso   
TIME_PERIOD                                                                  
2005-01-01                 -14.969892  269.701032   24.086022   253.052381   
2005-02-01                 -36.017857  266.743474   -9.425595   302.660095   
2005-03-01                  20.650538  266.528149   54.857527   332.319386   
2005-04-01                  78.601667  253.719444  113.894444   336.259203   
2005-05-01                 134.995161  247.859946  170.809140   315.863580   
...                               ...         ...         ...          ...   
2022-08-01                 218.941935  222.129167  243.446237   260.923571   
2022-09-01                 189.631034  221.270833  180.200000   260.132837   
2022-10-01                 172.066092  242.066667  138.333333   280.302304   
2022-11-01                 117.805769  238.194444   96.594444   271.984864   
2022-12-01                  88.849383  249.907407   49.658602   260.295430   

                                                                            \
PARTNER_Labels    Cameroon      Canada       Chile       China    Colombia   
TIME_PERIOD                                                                  
2005-01-01      265.058824         NaN  193.505177  -39.628008  227.893553   
2005-02-01      301.824074         NaN  194.394702  -25.939147  254.429167   
2005-03-01      328.450000 -104.471986  179.134537   50.029574  258.143750   
2005-04-01      310.826087  -20.064078  152.958690  132.430808  251.740190   
2005-05-01      290.272727   39.783135  138.762942  176.286936  251.447116   
...                    ...         ...         ...         ...         ...   
2022-08-01      231.275000  179.322868   98.776587  234.647872  248.137771   
2022-09-01      224.533333  1

## SAVE EXCEL FILE FULL DATAFRAME

In [298]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'SUNFLOWER_FULL_{now}.xlsx'
df_sunflower_total_updated.to_excel(filename, index=True)

## STANDARDIZATION

In [299]:
from sklearn.preprocessing import StandardScaler

df_sunflower_total_updated_std = df_sunflower_total_updated.copy()

# get a list of all the column names in the DataFrame
all_cols = list(df_sunflower_total_updated_std.columns)

# create a list of all columns to normalize, excluding 'price' for all countries
cols_to_normalize = [col for col in all_cols if 'price' not in col]

# instantiate the StandardScaler object
scaler = StandardScaler()

# fit the scaler on the selected columns
scaler.fit(df_sunflower_total_updated_std[cols_to_normalize])

# transform the selected columns using the scaler
df_sunflower_total_updated_std[cols_to_normalize] = scaler.transform(df_sunflower_total_updated_std[cols_to_normalize])
df_sunflower_total_updated_std

C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


AVG_TAVG                                                    \
PARTNER_Labels   Albania   Algeria    Angola   Armenia Australia   Austria   
TIME_PERIOD                                                                  
2005-01-01           NaN -1.625629  1.034987 -1.798018  1.252027 -1.083560   
2005-02-01           NaN -1.515783  0.895962 -1.792527  1.137852 -1.677014   
2005-03-01           NaN -0.678237 -3.652156 -1.326795  0.778492 -1.095545   
2005-04-01           NaN -0.223420 -0.216241 -0.337526  0.401334 -0.194828   
2005-05-01           NaN  0.555589 -1.447610  0.309276 -0.621650  0.507542   
...                  ...       ...       ...       ...       ...       ...   
2022-08-01     -0.353314  1.516497 -1.388027  1.611930 -1.166940  1.470106   
2022-09-01     -0.353314  1.167532 -1.388027  1.194183 -0.675527  0.435474   
2022-10-01     -0.353314  0.447761 -1.388027  0.481636 -0.073467  0.358105   
2022-11-01     -0.353314 -0.396555 -1.388027 -0.210411  0.216970 -0.531992   
2022-12-01     -0.353314 -0.631203 -1.388027 -0.815662  0.809199 -1.201236   

                                                         \
PARTNER_Labels Azerbaijan   Bahrain   Belarus   Belgium   
TIME_PERIOD                                               
2005-01-01            NaN -1.872269 -0.977534 -1.407486   
2005-02-01      -1.056052 -1.628760 -1.582130 -1.595379   
2005-03-01      -1.105865 -1.087113 -1.264649 -0.966726   
2005-04-01       0.425891 -0.373761  0.047158  0.002594   
2005-05-01       0.537971  0.333129  0.626557  0.323066   
...                   ...       ...       ...       ...   
2022-08-01       1.292319  1.213405  1.518355  1.797240   
2022-09-01       0.769955  1.000361  0.272778  0.968402   
2022-10-01       0.046010  0.393048  0.071877  0.640612   
2022-11-01      -0.748960 -0.110751 -0.685604 -0.393308   
2022-12-01      -1.487619 -0.900655 -1.225233 -1.298613   

                                                                        \
PARTNER_Labels Bosnia and Herzegovina    Brazil  Bulgaria Burkina Faso   
TIME_PERIOD                                                              
2005-01-01                  -1.828173  1.315208 -1.272341    -1.362267   
2005-02-01                  -2.108647  1.191699 -1.691326     0.632883   
2005-03-01                  -1.353515  1.182707 -0.887616     1.825736   
2005-04-01                  -0.581289  0.647810 -0.149497     1.984190   
2005-05-01                   0.170181  0.403115  0.562088     1.163908   
...                               ...       ...       ...          ...   
2022-08-01                   1.288810 -0.671414  1.470245    -1.045700   
2022-09-01                   0.898229 -0.707258  0.679499    -1.077502   
2022-10-01                   0.664168  0.161185  0.156054    -0.266315   
2022-11-01                  -0.058876 -0.000521 -0.365793    -0.600830   
2022-12-01                  -0.444733  0.488618 -0.952616    -1.070962   

                                                                  \
PARTNER_Labels  Cameroon    Canada     Chile     China  Colombia   
TIME_PERIOD                                                        
2005-01-01     -0.036075       NaN  1.463038 -1.586006 -0.527733   
2005-02-01      1.411027       NaN  1.489391 -1.443774  0.675055   
2005-03-01      2.459040 -1.295559  1.037288 -0.654437  0.843427   
2005-04-01      1.765352 -0.429196  0.261793  0.201736  0.553170   
2005-05-01      0.956359  0.185077 -0.158775  0.657414  0.539886   
...                  ...       ...       ...       ...       ...   
2022-08-01     -1.365827  1.617314 -1.343425  1.263801  0.389883   
2022-09-01     -1.631183  1.122685 -0.648089  0.844932  0.404835   
2022-10-01     -1.720400  0.521893  0.205445  0.062542 -0.086596   
2022-11-01     -1.809618 -0.600848  0.499614 -0.534900 -0.060458   
2022-12-01     -1.809618 -1.290403  1.053763 -1.512325  0.031090   

                                                                              \
PARTNER_Labels Congo,  Dem

## SAVE EXCEL FILE

### STANDARDIZED FULL DATA

In [300]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'SUNFLOWER_FULL_STANDARDIZED_{now}.xlsx'
df_sunflower_total_updated_std.to_excel(filename, index=True)

## ANALYSIS SUNFLOWER OIL IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [301]:
total = df_sunflower['OBS_VALUE_QUANTITY_TON'].sum()

total

8794710.3

In [302]:
df_filtered = df_sunflower

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)   

,index,PARTNER_Labels,OBS_VALUE_QUANTITY_TON,cumulative_sum,cumulative_sum_pct
0,82,Ukraine,4109777.1,4109777.1,0.467301
1,0,Argentina,984297.7,5094074.8,0.579220
2,26,Hungary,818426.8,5912501.6,0.672279
3,4,Belgium,796230.1,6708731.7,0.762814
4,23,Germany,672381.3,7381113.0,0.839267
5,22,France,643849.1,8024962.1,0.912476
6,62,Russia,132356.6,8157318.7,0.927526
7,8,Bulgaria,105809.5,8263128.2,0.939557
8,70,Spain,96798.9,8359927.1,0.950563
9,61,Romania,90881.6,8450808.7,0.960897


In [303]:
# Declare splitting point for cumulative import to NL history, biggest partners
splitting_point = 0.8

for i, row in grouped.iterrows():
    if row['cumulative_sum_pct'] >= splitting_point:
        top_x = i + 1
        break
        
top_partners = grouped['PARTNER_Labels'].head(top_x).tolist()
top_partners

['Ukraine', 'Argentina', 'Hungary', 'Belgium', 'Germany']

## FILTERING TOP PARTNERS

In [304]:
pd.set_option('display.max_columns', 90) #replace n with the number of columns you want to see completely

### FULL DATA

In [305]:
# filter the pivot table using the loc method
df_sunflower_total_filtered = df_sunflower_total_updated.loc[:, (slice(None), top_partners)]


# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_sunflower_total_filtered = df_sunflower_total_filtered.sort_index(axis=1)
df_sunflower_total_filtered.head(5)

AVG_TAVG                                     MAX_TMAX  \
PARTNER_Labels  Belgium     Germany     Hungary     Ukraine  Belgium   
TIME_PERIOD                                                            
2005-01-01        30.70   28.651592   -1.306452    0.869290     79.0   
2005-02-01        20.00   -1.962406  -25.371429  -38.376438    100.0   
2005-03-01        55.80   45.730051   38.383871   -3.794989    190.0   
2005-04-01       111.00  101.859649  112.460000   90.622214    200.0   
2005-05-01       129.25  134.667233  162.993548  161.493475    223.0   

                                                   MIN_TMIN              \
PARTNER_Labels     Germany     Hungary     Ukraine  Belgium     Germany   
TIME_PERIOD                                                               
2005-01-01      129.842105  137.000000  102.142857    -31.0  -91.000000   
2005-02-01      108.315789   57.666667   60.142857    -66.0 -125.684211   
2005-03-01      180.894737  209.000000  112.468208    -48.0  -99.526316   
2005-04-01      213.315789  218.000000  204.189944     57.0  -12.421053   
2005-05-01      314.684211  321.000000  307.400000     53.0   22.631579   

                                       OBS_VALUE_QUANTITY_TON                  \
PARTNER_Labels     Hungary     Ukraine              Argentina Belgium Germany   
TIME_PERIOD                                                                     
2005-01-01     -136.000000 -105.930233             21086.3000  1660.8  1309.8   
2005-02-01     -167.600000 -190.275862             18325.9000   984.8  1714.5   
2005-03-01     -126.400000 -123.523810             17003.9625  1254.0  1936.2   
2005-04-01       -2.000000  -37.809524             15682.0250  2300.6   983.5   
2005-05-01       27.833333   57.571429             14360.0875  1137.9  2566.8   

                                  OBS_VALUE_VALUE_1000EURO                    \
PARTNER_Labels    Hungary Ukraine                Argentina  Belgium  Germany   
TIME_PERIOD                                                                    
2005-01-01            NaN     NaN             10489.970000  1253.70  1018.98   
2005-02-01      23.600000     NaN              8946.330000   729.94  1317.62   
2005-03-01      22.366667     NaN             12741.530000   935.68  1508.41   
2005-04-01      21.133333     NaN             16665.380000  1626.79   782.74   
2005-05-01      19.900000     NaN             15562.506667   824.81  1827.82   

                                  OIL_PRICE $/bbl                        \
PARTNER_Labels    Hungary Ukraine       Argentina    Belgium    Germany   
TIME_PERIOD                                                               
2005-01-01            NaN     NaN       42.972278  42.972278  42.972278   
2005-02-01      23.560000     NaN       44.818211  44.818211  44.818211   
2005-03-01      22.300556     NaN       50.942879  50.942879  50.942879   
2005-04-01      21.041111     NaN       50.640476  50.640476  50.640476   
2005-05-01      19.781667     NaN       47.826573  47.826573  47.826573   

                                     Sunflower Seed Price Futures          \
PARTNER_Labels    Hungary    Ukraine                    Argentina Belgium   
TIME_PERIOD                                                                 
2005-01-01      42.972278  42.972278                          NaN     NaN   
2005-02-01      44.818211  44.818211                          NaN     NaN   
2005-03-01      50.942879  50.942879                          NaN     NaN   
2005-04-01      50.640476  50.640476                          NaN     NaN   
2005-05-01      47.826573  47.826573                          NaN     NaN   

                                       agricultural_land           \
PARTNER_Labels Germany Hungary Ukraine         Argentina  Belgium   
TIME_PERIOD                                                         
2005-01-01         NaN     NaN     NaN      1.299209e+06  13830.0   
2005-02-01         NaN     NaN     NaN      1.299391e

#### SAVE EXCEL FILE FILTERED NON STANDARDIZED

In [306]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'SUNFLOWER_FILTERED_{now}.xlsx'
df_sunflower_total_filtered.to_excel(filename, index=True)

### STANDARDIZED DATA

In [307]:
# filter the pivot table using the loc method
df_sunflower_total_filtered_std = df_sunflower_total_updated_std.loc[:, (slice(None), top_partners)]

# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_sunflower_total_filtered_std = df_sunflower_total_filtered_std.sort_index(axis=1)
df_sunflower_total_filtered_std.head(5)

AVG_TAVG                                MAX_TMAX            \
PARTNER_Labels   Belgium   Germany   Hungary   Ukraine   Belgium   Germany   
TIME_PERIOD                                                                  
2005-01-01     -1.407486 -1.169212 -1.484828 -1.061958 -1.733743 -1.103121   
2005-02-01     -1.595379 -1.641851 -1.789502 -1.499566 -1.466596 -1.369811   
2005-03-01     -0.966726 -0.905543 -0.982329 -1.113967 -0.321683 -0.470627   
2005-04-01      0.002594 -0.038976 -0.044488 -0.061172 -0.194470 -0.068961   
2005-05-01      0.323066  0.467529  0.595291  0.729075  0.098119  1.186897   

                                    MIN_TMIN                                \
PARTNER_Labels   Hungary   Ukraine   Belgium   Germany   Hungary   Ukraine   
TIME_PERIOD                                                                  
2005-01-01     -1.170753 -1.144196 -0.972661 -1.450704 -1.797334 -0.943533   
2005-02-01     -2.093786 -1.582297 -1.622534 -1.991585 -2.199518 -1.794689   
2005-03-01     -0.333042 -1.036493 -1.288313 -1.583667 -1.675152 -1.121075   
2005-04-01     -0.228328 -0.079748  0.661306 -0.225310 -0.091872 -0.256108   
2005-05-01      0.970064  0.996832  0.587035  0.321316  0.287827  0.706408   

               OBS_VALUE_QUANTITY_TON                                        \
PARTNER_Labels              Argentina   Belgium   Germany   Hungary Ukraine   
TIME_PERIOD                                                                   
2005-01-01                   3.135487 -1.135016 -0.930556       NaN     NaN   
2005-02-01                   2.618829 -1.506825 -0.724938 -0.637433     NaN   
2005-03-01                   2.371405 -1.358761 -0.612298 -0.637650     NaN   
2005-04-01                   2.123981 -0.783118 -1.096341 -0.637868     NaN   
2005-05-01                   1.876557 -1.422618 -0.291906 -0.638086     NaN   

               OBS_VALUE_VALUE_1000EURO                                        \
PARTNER_Labels                Argentina   Belgium   Germany   Hungary Ukraine   
TIME_PERIOD                                                                     
2005-01-01                     1.836416 -1.288825 -0.979460       NaN     NaN   
2005-02-01                     1.453138 -1.536744 -0.841048 -0.630515     NaN   
2005-03-01                     2.395467 -1.439358 -0.752622 -0.630792     NaN   
2005-04-01                     3.369738 -1.112225 -1.088950 -0.631070     NaN   
2005-05-01                     3.095901 -1.491838 -0.604585 -0.631347     NaN   

               OIL_PRICE $/bbl                                          \
PARTNER_Labels       Argentina   Belgium   Germany   Hungary   Ukraine   
TIME_PERIOD                                                              
2005-01-01           -1.267942 -1.267942 -1.267942 -1.267942 -1.267942   
2005-02-01           -1.190552 -1.190552 -1.190552 -1.190552 -1.190552   
2005-03-01           -0.933780 -0.933780 -0.933780 -0.933780 -0.933780   
2005-04-01           -0.946458 -0.946458 -0.946458 -0.946458 -0.946458   
2005-05-01           -1.064429 -1.064429 -1.064429 -1.064429 -1.064429   

               Sunflower Seed Price Futures                                  \
PARTNER_Labels                    Argentina Belgium Germany Hungary Ukraine   
TIME_PERIOD                                                                   
2005-01-01                              NaN     NaN     NaN     NaN     NaN   
2005-02-01                              NaN     NaN     NaN     NaN     NaN   
2005-03-01                              NaN     NaN     NaN     NaN     NaN   
2005-04-01                              NaN     NaN     NaN     NaN     NaN   
2005-05-01                              NaN     NaN     NaN     NaN     NaN   

               agricultural_land                                          \
PARTNER_Labels         Argentina   Belgium   Germany   Hungary   Ukraine   
TIME_PERIOD                                                                
2005-01-01              1.3648

#### SAVE EXCEL FILE FILTERED STANDARDIZED

In [308]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'SUNFLOWER_FILTERED_STANDARDIZED_{now}.xlsx'
df_sunflower_total_filtered_std.to_excel(filename, index=True)